In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
USE CATALOG adventureworks;
USE SCHEMA gold;

In [0]:
df_product = spark.table("adventureworks.silver.silver_product")
df_region = spark.table("adventureworks.silver.silver_region")
df_reseller = spark.table("adventureworks.silver.silver_reseller")
df_sales = spark.table("adventureworks.silver.silver_sales")
df_salesperson = spark.table("adventureworks.silver.silver_salesperson")
df_salespersonregion = spark.table("adventureworks.silver.silver_salespersonregion")
df_targets = spark.table("adventureworks.silver.silver_targets")

### Transforming silver_product


In [0]:
display(df_product)

product_key,product,standard_cost,color,subcategory,category
210,"HL Road Frame - Black, 58",868.63,Black,Road Frames,Components
215,"Sport-100 Helmet, Black",12.03,Black,Helmets,Accessories
216,"Sport-100 Helmet, Black",13.88,Black,Helmets,Accessories
217,"Sport-100 Helmet, Black",13.09,Black,Helmets,Accessories
253,"LL Road Frame - Black, 58",176.2,Black,Road Frames,Components
254,"LL Road Frame - Black, 58",170.14,Black,Road Frames,Components
255,"LL Road Frame - Black, 58",204.63,Black,Road Frames,Components
256,"LL Road Frame - Black, 60",176.2,Black,Road Frames,Components
257,"LL Road Frame - Black, 60",170.14,Black,Road Frames,Components
258,"LL Road Frame - Black, 60",204.63,Black,Road Frames,Components


In [0]:
df_product = df_product.withColumn('model_name',trim(split(col('product'),',')[0]))

In [0]:
color_pattern = '(' + 'Black|Red|Silver|Yellow|Blue' + ')'

df_product = (df_product
              
    # Remove color names
    .withColumn("model_name_clean", regexp_replace(col("model_name"), color_pattern, ""))

    # Remove '-W' and '-W -'
    .withColumn("model_name_clean", regexp_replace(col("model_name_clean"), r'-W\s*-?', ""))

    # Remove multiple spaces or leftover hyphens at the end
    .withColumn("model_name_clean", regexp_replace(col("model_name_clean"), r'[-\s]+$', ""))

    # Remove extra internal spaces
    .withColumn("model_name_clean", regexp_replace(col("model_name_clean"), r'\s+', " "))

    # Trim leading/trailing spaces
    .withColumn("model_name_clean", trim(col("model_name_clean")))
)

In [0]:
df_product = df_product.drop('model_name')

In [0]:
df_product = df_product.withColumnRenamed('model_name_clean', 'model_name')

### Transforming silver_sales


In [0]:
df_sales = df_sales.withColumn("profit", round(col("sales") - col("cost"),2))

In [0]:
df_sales = df_sales.withColumn("loss",when(col("profit")<0, True).otherwise(False))

In [0]:
df_sales = df_sales.withColumn("profir_category", when(col("profit")<0, "Loss")
                                                 .when(col("profit")>=0, "Low Profit")
                                                 .when(col("profit")>=50, "Medium Profit")
                                                 .when(col("profit")>=100, "High Profit")
                                                 .when(col("profit")>=500, "Very High Profit")
                                                 .otherwise("Highest Profit"))

### Transforming silver_salesperson

In [0]:
df_salesperson = df_salesperson.withColumn("username",regexp_replace(split(col("email"), "@")[0], "-", "."))

### Writing all the files to Gold Layer

In [0]:
def write_to_gold_tables(df,table_name):
    df.write.format('delta')\
            .mode('overwrite')\
            .option('overwriteSchema','true')\
            .saveAsTable(f"adventureworks.gold.{table_name}")

In [0]:
write_to_gold_tables(df_product,'gold_product')
write_to_gold_tables(df_region,'gold_region')
write_to_gold_tables(df_reseller,'gold_reseller')
write_to_gold_tables(df_sales,'gold_sales')
write_to_gold_tables(df_salesperson,'gold_salesperson')
write_to_gold_tables(df_salespersonregion,'gold_salespersonregion')
write_to_gold_tables(df_targets,'gold_targets')

### Writing all the files in Delta Format to Gold Container in ADLS

In [0]:
gold_path = 'abfss://gold@adventureworksdls001.dfs.core.windows.net/'

df_product.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}product")
df_region.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}region")
df_reseller.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}reseller")
df_sales.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}sales")
df_salesperson.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}salesperson")
df_salespersonregion.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}salespersonregion")
df_targets.write.format('delta').mode('overwrite').option('overwriteSchema','true').save(f"{gold_path}targets")